In [61]:
import sympy as sp
from geometry_msgs.msg import Point

In [62]:
# Variáveis
θ1, θ2 = sp.symbols("θ1, θ2")
λ = sp.symbols("λ")
Δ = sp.symbols("Δ")
v = sp.symbols("v")
L_PUN = sp.symbols('L_PUN')
x_goal, y_goal, z_goal = sp.symbols('x_goal y_goal z_goal')

In [35]:
# Rotação do corpo em torno de yaw (λ)
q_corpo = sp.Quaternion(sp.cos(λ)/2, 0, 0, sp.sin(λ)/2) # Quaternion de rotação do corpo em yaw
display(q_corpo)

cos(λ)/2 + 0*i + 0*j + sin(λ)/2*k

In [36]:
# Rotação do ombro em torno de pitch (θ1)
q_ombro = sp.Quaternion(sp.cos(θ1)/2, 0, sp.sin(θ1)/2, 0) # Quaternion de rotação do ombro
display(q_ombro)

cos(θ1)/2 + 0*i + sin(θ1)/2*j + 0*k

In [37]:
# Rotação do cotovelo em torno de pitch (θ2)
q_cotovelo = sp.Quaternion(sp.cos(θ2)/2, 0, sp.sin(θ2)/2, 0) # Quaternion de rotação do cotovelo
display(q_cotovelo)

cos(θ2)/2 + 0*i + sin(θ2)/2*j + 0*k

In [38]:
# Rotação do braço (ombro + cotovelo)
q_braço = q_ombro * q_cotovelo # Quaternion de rotação do braço (sem punho)
q_braço = q_braço.simplify()
display(q_braço)

cos(θ1 + θ2)/4 + 0*i + sin(θ1 + θ2)/4*j + 0*k

In [39]:
# Rotação completa (corpo em yaw + ajuste do braço)
q_completo = q_corpo * q_braço
display(q_completo)

cos(λ)*cos(θ1 + θ2)/8 + (-sin(λ)*sin(θ1 + θ2)/8)*i + sin(θ1 + θ2)*cos(λ)/8*j + sin(λ)*cos(θ1 + θ2)/8*k

In [41]:
# Rotação de Δ em torno de pitch
q_Δ = sp.Quaternion(sp.cos(Δ/2), 0, sp.sin(Δ/2), 0)
display(q_Δ)

cos(Δ/2) + 0*i + sin(Δ/2)*j + 0*k

In [67]:
# Considerando que q_braço = q(Δ, <0, 1, 0>)
eq_Δ = sp.Eq(q_braço, q_Δ).simplify()
display(eq_Δ)

Eq(cos(Δ/2) + 0*i + sin(Δ/2)*j + 0*k, cos(θ1 + θ2)/4 + 0*i + sin(θ1 + θ2)/4*j + 0*k)

In [68]:
# Equações obtidas de q_braço = q(Δ, <0, 1, 0>)
w_L, y_L = eq_Δ.lhs.a, eq_Δ.lhs.c
w_R, y_R = eq_Δ.rhs.a, eq_Δ.rhs.c
eq_Δ_w = sp.Eq(w_L, w_R)
eq_Δ_x = sp.Eq(y_L, y_R)
display(eq_Δ_w, eq_Δ_x)

Eq(cos(Δ/2), cos(θ1 + θ2)/4)

Eq(sin(Δ/2), sin(θ1 + θ2)/4)

In [50]:
# v é o vetor que aponta para o offsetGoal
v = sp.sqrt(z_goal**2 + (x_goal-L_PUN)**2)
display(v)

sqrt(z_goal**2 + (-L_PUN + x_goal)**2)

In [51]:
# Considerando que Δ = asin(goal.z, v)
val_Δ = sp.asin(z_goal / v)
display(val_Δ)

asin(z_goal/sqrt(z_goal**2 + (-L_PUN + x_goal)**2))

In [53]:
# Sistema
system = (eq_Δ_w, eq_Δ_x)
# system = (eq_Δ_w.subs(Δ, val_Δ).simplify(), eq_Δ_x.subs(Δ, val_Δ).simplify())
display(system[0], system[1])

Eq(cos(Δ/2), cos(θ1 + θ2)/4)

Eq(sin(Δ/2), sin(θ1 + θ2)/4)

In [17]:
# Cinemática direta
# set_OMB = 30
# set_COT = 60
# sub = [None, None]
# sub[0] = system[0].subs(θ1, set_OMB).subs(θ2, set_COT)
# sub[1] = system[1].subs(θ1, set_OMB).subs(θ2, set_COT)
# display(sub[0], sub[1])

In [66]:
# Cinemática inversa
length_OMB = 23.5
length_COT = 24.5
length_PUN = 6.5
goal = Point(39.101, 0.0, 32.968)
sub = [None, None]
sub[0] = system[0].subs(Δ, val_Δ).subs(z_goal, goal.z).subs(x_goal, goal.x).subs(L_PUN, length_PUN)
sub[1] = system[1].subs(Δ, val_Δ).subs(z_goal, goal.z).subs(x_goal, goal.x).subs(L_PUN, length_PUN)
display(sub[0], sub[1])
sol = sp.solve((sub[0], sub[1]), (θ1, θ2))
display(sol)

Eq(0.92280495757596, cos(θ1 + θ2)/4)

Eq(0.38526745291191, sin(θ1 + θ2)/4)

[]

In [64]:
''' 
    TODO
    - Corrigir absurdo na cinemática inversa
    - Funçoes do sympy pra construir os quaternions
'''

' \n    TODO\n    - Corrigir absurdo na cinemática inversa\n    - Funçoes do sympy pra construir os quaternions\n'